In [1]:
from radfusion3.data import DataModule
from omegaconf import OmegaConf
import yaml

/usr/local/lib/python3.9/dist-packages/wandb/apis/public.py:3106: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
from hydra import initialize, compose
from omegaconf import OmegaConf

# 第一步：初始化 config 路径
initialize(config_path="./radfusion3/configs")  # 注意这里要根据你 notebook 的位置调整路径

# 第二步：合成配置，相当于你 CLI 中的命令参数
cfg = compose(
    config_name="classify",  # 对应 run_classify.py 里的 config_name
    overrides=[
        "model=resnetv2",
        "dataset=rsna",
        "dataset.transform.final_size=224"
    ]
)

# 第三步：确认 dataset 配置存在
print(cfg.dataset.name if "name" in cfg.dataset else cfg.dataset)


/tmp/ipykernel_675323/3718602523.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="./radfusion3/configs")  # 注意这里要根据你 notebook 的位置调整路径


{'type': 'rsna', 'csv_path': '/projects/eclarson/stems/STEMC/EHR/INSPECT/RSNA/train_with_split.csv', 'sample_frac': 1.0, 'dicom_dir': '/projects/eclarson/stems/STEMC/EHR/INSPECT/RSNA/train', 'output_dir': '/projects/eclarson/stems/STEMC/EHR/INSPECT/RSNA/rsna_features', 'test_split': 'all', 'target': 'pe_present_on_image', 'contextualize_slice': False, 'sample_strategy': 'fix', 'num_slices': 150, 'feature_size': 768, 'weighted_sample': True, 'batch_size': 32, 'transform': {'channels': 'repeat', 'resize_size': 256, 'crop_size': 224, 'final_size': 224}, 'pretrain_args': {'model_type': 'resnetv2_101_ct', 'input_size': 256, 'channel_type': 'window'}}


In [8]:
from radfusion3.data import DataModule

# 改为 "train"、"valid"、或你 CSV 中存在的 split
dm = DataModule(cfg, test_split="val")  

# 获取 DataLoader
val_loader = dm.test_dataloader()

# 检查是否成功加载
print("验证集样本数:", len(val_loader.dataset))

for batch in val_loader:
    print("✔️ 成功获取一个 batch")
    if isinstance(batch, dict):
        for k, v in batch.items():
            print(f"{k}: {v.shape if hasattr(v, 'shape') else type(v)}")
    else:
        print("类型:", type(batch))
    break

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


pe_present_on_image
0    97586
1    20268
Name: count, dtype: int64
验证集样本数: 117854
✔️ 成功获取一个 batch
类型: <class 'list'>


In [5]:
dm = DataModule(cfg, test_split="train")  

# 获取 DataLoader
val_loader = dm.test_dataloader()

# 检查是否成功加载
print("验证集样本数:", len(val_loader.dataset))

pe_present_on_image
0    338464
1     67543
Name: count, dtype: int64
验证集样本数: 406007


In [6]:
dm = DataModule(cfg, test_split="test")  

# 获取 DataLoader
val_loader = dm.test_dataloader()

# 检查是否成功加载
print("验证集样本数:", len(val_loader.dataset))

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


pe_present_on_image
0    46859
1     8729
Name: count, dtype: int64
验证集样本数: 55588


In [7]:
dm = DataModule(cfg, test_split="all")  

# 获取 DataLoader
val_loader = dm.test_dataloader()

# 检查是否成功加载
print("验证集样本数:", len(val_loader.dataset))

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


pe_present_on_image
0    482909
1     96540
Name: count, dtype: int64
验证集样本数: 579449


In [5]:
import pandas as pd

df = pd.read_csv('/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/sample/study_mapping_20250611.tsv', sep='\t')

In [6]:
df

,impression_id,person_id,procedure_occurrence_id,procedure_DATETIME,note_id,note_DATETIME,image_id,provenance
0,647,124887787,78357477.0,2010-08-28T18:01,207996501.0,2010-08-29T15:04,PEc38881,radfusion
1,682,125200548,61918785.0,2002-05-16T06:36,217943432.0,2002-05-17T10:56,PEc34873,radfusion
2,1345,127612575,65495089.0,2005-01-09T15:49,201694100.0,2005-01-09T20:50,PEc356c4,radfusion
3,1382,124842110,NaN,2006-02-22T13:06,194115044.0,2006-02-23T11:13,PEc347ed,radfusion
4,2173,127561215,NaN,NaN,NaN,NaN,PEc378af,INSPECT
...,...,...,...,...,...,...,...,...
95,22433,125705821,NaN,2004-11-25T22:17,176279159.0,2004-11-30T00:21,PEc38723,radfusion
96,22551,128049235,76730903.0,2010-03-26T10:52,198035616.0,2010-03-30T16:09,PEc3eac5,radfusion
97,22740,124636336,116355808.0,2016-02-28T16:58,195536531.0,2016-02-28T19:42,PEc3eb9e,INSPECT
98,22863,126778181,69688394.0,2007-07-08T01:45,207634958.0,2007-07-11T15:51,PEc3722e,radfusion


In [7]:
import nibabel as nib

/usr/local/lib/python3.9/dist-packages/pylibjpeg/utils.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import iter_entry_points


In [10]:
import h5py

with h5py.File("/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/features.hdf5", "r") as f:
    print(list(f.keys()))  # 应该输出数据集的键名，例如 ["impression_id", "features", ...]


['PE1677746', 'PE1677747', 'PE1677855', 'PE1677856', 'PE1677858', 'PE1677859', 'PE167785e', 'PE167785f', 'PE1677860', 'PE1677861', 'PE1677865', 'PE1677867', 'PE1677868', 'PE167786b', 'PE167786d', 'PE167786e', 'PE167786f', 'PE1677870', 'PE1677873', 'PE1677876', 'PE1677877', 'PE1677879', 'PE167787c', 'PE167787d', 'PE167787e', 'PE1677882', 'PE1677885', 'PE1677887', 'PE1677888', 'PE1677889', 'PE167788b', 'PE167788f', 'PE1677890', 'PE1677891', 'PE1677892', 'PE1677893', 'PE1677894', 'PE1677896', 'PE1677898', 'PE167789a', 'PE167789b', 'PE167789c', 'PE167789d', 'PE167789e', 'PE167789f', 'PE16778a0', 'PE16778a1', 'PE16778a5', 'PE16778a6', 'PE16778a7', 'PE16778a8', 'PE16778a9', 'PE16778aa', 'PE16778af', 'PE16778b0', 'PE16778b1', 'PE16778b2', 'PE16778b3', 'PE16778b4', 'PE16778b5', 'PE16778b6', 'PE16778b7', 'PE16778bb', 'PE16778bc', 'PE16778bd', 'PE16778bf', 'PE16778c0', 'PE16779d5', 'PE16779d6', 'PE16779d7', 'PE16779da', 'PE16779de', 'PE16779df', 'PE16779e0', 'PE16779e1', 'PE16779e3', 'PE16779e4'

In [5]:
import os

path = "/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/sample/pre_features"  # 替换为你的路径
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(f"✅ 当前目录文件数: {len(files)}")

✅ 当前目录文件数: 101


In [13]:
path = "/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/pre_features"  # 替换为你的路径
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(f"✅ 当前目录文件数: {len(files)}")

✅ 当前目录文件数: 14384


In [15]:
import os

# 设置你要检查的文件路径
file_path = '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f310c.nii.gz'

# 检查文件是否存在
if os.path.isfile(file_path):
    print(f"✅ 文件存在: {file_path}")
else:
    print(f"❌ 文件不存在或无权限访问: {file_path}")


❌ 文件不存在或无权限访问: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f310c.nii.gz


In [18]:
import pandas as pd
import os

# TSV 文件路径
tsv_path = '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/study_mapping_20250611.tsv'

# 文件夹路径
nii_dir = '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA'

# 读取 TSV 文件
df = pd.read_csv(tsv_path, sep='\t')
print(f"✅ 成功读取 {tsv_path}，共有 {len(df)} 行")

# 检查 image_id 列是否存在
if 'image_id' not in df.columns:
    print("❌ 错误：找不到 'image_id' 列")
else:
    # 检查文件是否存在
    missing_ids = []
    for image_id in df['image_id']:
        nii_path = os.path.join(nii_dir, f"{image_id}.nii.gz")
        if not os.path.isfile(nii_path):
            missing_ids.append(image_id)

    print(f"\n🔍 缺失的 image_id 数量: {len(missing_ids)}")
    if missing_ids:
        print("❌ 以下 image_id 的 .nii.gz 文件不存在：")
        for i in missing_ids:
            print(i)
    else:
        print("✅ 所有 image_id 对应的 .nii.gz 文件都存在")


✅ 成功读取 /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/study_mapping_20250611.tsv，共有 23248 行

🔍 缺失的 image_id 数量: 8
❌ 以下 image_id 的 .nii.gz 文件不存在：
PE45241fd
PE87cf6e
PE45283bf
PE876fb7
PE9f749e
PE9f310c
PE45295ec
PE452890e


/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23248/23248 [00:01<00:00, 13212.41it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
Testing: 0it [00:00, ?it/s][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f749e.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f749e.nii.gz'
Testing DataLoader 0:   9%|██████████████▌                                                                                                                                                       | 16/182 [00:27<04:46,  1.72s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f310c.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE9f310c.nii.gz'
Testing DataLoader 0:  26%|███████████████████████████████████████████▊                                                                                                                          | 48/182 [05:43<15:57,  7.15s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45283bf.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45283bf.nii.gz'
Testing DataLoader 0:  27%|████████████████████████████████████████████▋                                                                                                                         | 49/182 [07:57<21:36,  9.75s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE87cf6e.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE87cf6e.nii.gz'
Testing DataLoader 0:  62%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                               | 112/182 [16:20<10:13,  8.76s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE452890e.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE452890e.nii.gz'
Testing DataLoader 0:  70%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                 | 128/182 [18:55<07:58,  8.87s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45295ec.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45295ec.nii.gz'
[ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45241fd.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE45241fd.nii.gz'
Testing DataLoader 0:  79%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 144/182 [21:29<05:40,  8.96s/it][ERROR] Failed to read NIfTI: /projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE876fb7.nii.gz | No such file or no access: '/projects/eclarson/stems/STEMC/EHR/INSPECT/IMAGE/inpsect/inspectamultimodaldatasetforpulmonaryembolismdiagnosisandprog-3/full/CTPA/PE876fb7.nii.gz'
Testing DataLoader 0:  80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                            | 145/182 [23:34<06:00,  9.76s/it]Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [29:08<00:00,  9.61s/it]Processed 23248 instances with 6144 features
Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [30:31<00:00, 10.06s/it]